# My test

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
f_name = 'co2020_challenge_r200d15_5'
f1 = f_name + '.txt'
f2 = f_name + '.sol.txt'

sys.path.append('./ParticipantsSuite')
from InstanceCVRPTWUI import InstanceCVRPTWUI
from SolutionCVRPTWUI import SolutionCVRPTWUI
instance = InstanceCVRPTWUI(f1)

In [ ]:
if instance.isValid():
    instance.calculateDistances()

import numpy
nLoc = len(instance.Coordinates)
d_df = pd.DataFrame([ [ instance.calcDistance[i][j] for j in range(nLoc) ] for i in range(nLoc) ])

t_df = pd.DataFrame([(i.ID, i.weight, i.amount, i.cost) for i in instance.Tools], columns=['ID','weight','amount','cost']).set_index('ID')
r_df = pd.DataFrame([(i.ID, i.node, i.fromDay, i.toDay, i.numDays, i.tool, i.toolCount) for i in instance.Requests], columns=['ID','node','fromDay','toDay','numDays','tool','toolCount']).set_index('ID')
c_df = pd.DataFrame([(i.ID, i.X, i.Y) for i in instance.Coordinates], columns=['ID','X','Y']).set_index('ID')

In [ ]:
r_df_m = r_df.merge(c_df, left_on='node', right_index=True)
r_df_m = r_df_m.merge(t_df, left_on='tool', right_index=True)
r_df_m['size'] = r_df_m.toolCount * r_df_m.weight
r_df_m.drop(columns=['weight','amount','cost'], axis=1, inplace=True)

# initialize scheduled delivery and pickup by fromDay
r_df_m['scheduled_d'] = r_df_m.fromDay
r_df_m['scheduled_p'] = r_df_m.fromDay + r_df_m.numDays
r_df_schedule = r_df_m.copy()

In [ ]:
r_df_schedule[r_df_schedule.scheduled_d==1].sort_values(['scheduled_d','scheduled_p','node']).head()

In [ ]:
inventory_df = t_df.copy()
inventory_df['in_use'] = 0
inventory_df['used'] = 0
inventory_df.drop(columns=['weight','cost'])

In [ ]:
r_df_schedule = r_df_m.copy()

inventory_df = t_df.copy()
inventory_df['in_use'] = 0
inventory_df['used'] = 0
inventory_df.drop(columns=['weight','cost'])

last_day = (r_df_schedule['toDay'] + r_df_schedule['numDays']).max()
for c_day in range(last_day):
    # i+1
    df_d = r_df_schedule[r_df_schedule.scheduled_d == (c_day+1)].sort_values(['toDay','numDays'])
    df_p = r_df_schedule[r_df_schedule.scheduled_p == (c_day+1)]

    #print("inventory start day {}".format(c_day+1))
    #print(inventory_df[['amount','in_use','used']])
    print("\n")
    print("Day: {}".format(c_day))

    d_ids = []
    p_ids = []

    for i,r in df_d.iterrows():
    
        # if there is inventory
        old_stock = inventory_df.loc[r.tool,'amount']
        new_stock = old_stock - r.toolCount
        if (new_stock) >= 0:
            #if r.tool == 4:
            #    print("{} took {}".format(i,r.toolCount))
            inventory_df.loc[r.tool,'amount'] = new_stock
            inventory_df.loc[r.tool, 'in_use'] += r.toolCount 
            r_df_schedule.loc[i, 'scheduled_p'] = r.scheduled_d + r.numDays
            d_ids.append(i)
        else:
            # toDay condition!!!
            if (r.scheduled_d + 1) > r.toDay:
                print("WARNING: too late delivery")
            old_day = r_df_schedule.loc[i, 'scheduled_d']
            new_day = old_day + 1
            r_df_schedule.loc[i, 'scheduled_d'] = new_day
            print("Note: rescheduled delivery from {} -> {}".format(old_day,r_df_schedule.loc[i, 'scheduled_d']))      

    for i,r in df_p.iterrows():
        if r.tool == 4:
            c4 += r.toolCount

        p_ids.append(i)

        # add picked-up goods to inventory
        old_stock = inventory_df.loc[r.tool,'amount']
        new_stock = old_stock + r.toolCount
        inventory_df.loc[r.tool,'amount'] = new_stock
        inventory_df.loc[r.tool, 'in_use'] -= r.toolCount 

    for i,r in inventory_df.iterrows(): 
        if r.in_use > r.used:
              inventory_df.loc[i, 'used'] = r.in_use

    #print("delivery ids: {}".format(d_ids))
    #print("pickup ids: {}".format(p_ids))

#r_df_schedule.sort_values(['scheduled_d','scheduled_p','node'])


In [ ]:
route_df = r_df_schedule.copy()
route_df = route_df.merge(d_df[0], left_on='node', right_index=True)
route_df.rename(columns={0:'distance'}, inplace=True)
route_df['distance'] = route_df['distance']**2
route_df.reset_index(inplace=True)

route_d_df = route_df[['scheduled_d','ID','distance']].copy()
route_d_df['route'] = route_df['ID'].apply(lambda x: [0,x,0])
route_d_df.rename(columns={'scheduled_d':'day'}, inplace=True)

route_p_df = route_df[['scheduled_p','ID','distance']].copy()
route_p_df['route'] = route_df['ID'].apply(lambda x: [0,-x,0])
route_p_df.rename(columns={'scheduled_p':'day'}, inplace=True)

route_df = pd.concat([route_d_df,route_p_df])

route_df.sort_values('day')

In [ ]:
with open(f2, 'w') as sol:

    sol.write("{} = {}\n".format(instance.LANG.TXT.dataset, instance.Dataset))
    sol.write("{} = {}\n".format(instance.LANG.TXT.name, instance.Name))

    days = route_df.day.sort_values().unique()
    for d in days:
        sol.write("\n")

        vehicles = route_df[route_df.day == d]
        no_vehicles = len(vehicles.index)

        sol.write("DAY = {}\n".format(d))
        sol.write("NUMBER_OF_VEHICLES = {}\n".format(no_vehicles))

        vec = 1
        for i,r in vehicles.iterrows():
            route = "\t".join(map(str,r.route))
            sol.write("{}\tR\t{}\n".format(vec,route))
            vec += 1
    sol.close()

In [ ]:
solution = SolutionCVRPTWUI(f2,instance,continueOnErr=True)
if not solution.isValid():
    print('errors:')
    print('\n'.join(solution.errorReport))
    print('warnings:')
    print('\n'.join(solution.warningReport))
solution.writeSolution(f2,True)